Importing Libraries

In [1]:
import os
import pickle
import warnings
from glob import glob

import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import (
    RandomForestRegressor,
    AdaBoostRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
)
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
import lightgbm as lgb
import xgboost as xgb
import re

Loading models and Data and converting the dependent variable

In [2]:
# -----------------------------
# Config
# -----------------------------
# METHOD_FOLDERS = ["AUTOFEAT", "FT", "OPENFE", "SAFE"]  # expected subfolders
METHOD_FOLDERS = ["BASIC","AUTOFEAT", "FT", "OPENFE", "SAFE"]  # expected subfolders
PREDICTIONS_DIR = "Predictions"  # where we write pickles with predictions
os.makedirs(PREDICTIONS_DIR, exist_ok=True)

RANDOM_STATE = 42

# -----------------------------
# Define models (with sensible scaling where needed)
# -----------------------------
def build_models():
    # Define regression models
    regression_models = {
        "Linear Regression": LinearRegression(),
        "KNN Regression": KNeighborsRegressor(),
        "SVM Regression": SVR(),
        "Random Forest Regression": RandomForestRegressor(random_state=42),
        "AdaBoost Regression": AdaBoostRegressor(random_state=42),
        "MLP Regression": MLPRegressor(random_state=42),
        "Decision Tree Regression": DecisionTreeRegressor(random_state=42),
        "Extremely Randomized Trees Regression": ExtraTreesRegressor(random_state=42),
        "Gradient Boosting Regression": GradientBoostingRegressor(random_state=42),
        "LightGBM Regression": lgb.LGBMRegressor(random_state=42,verbose = -1),
        "XGBoost Regression": xgb.XGBRegressor(random_state=42)
    }

    return regression_models

# def ensure_1d_target(y):
#     # Make y a 1-D vector
#     if isinstance(y, pd.DataFrame):
#         if y.shape[1] != 1:
#             raise ValueError(f"y must be 1-D; got shape {y.shape}")
#         y = y.iloc[:, 0]
#         print('IT HIT HERE')
#     return np.ravel(y)


# -----------------------------
# Utilities
# -----------------------------
def load_method_datasets(method_folder):
    """
    Read all *.pkl files under method_folder.
    Each file is assumed to be: { 'fold1': {...}, ..., 'fold5': {...} }.
    Returns dict: dataset_name -> folds_dict
    """
    path_pattern = os.path.join(method_folder, "*.pkl")
    files = sorted(glob(path_pattern))
    data = {}
    for fp in files:
        dataset_name = os.path.splitext(os.path.basename(fp))[0]
        with open(fp, "rb") as f:
            try:
                obj = pickle.load(f)
            except EOFError:
                warnings.warn(f"EOFError reading {fp}; skipping.")
                continue
        # basic sanity check
        if not isinstance(obj, dict):
            warnings.warn(f"Unexpected content in {fp} (not a dict); skipping.")
            continue
        expected_folds = {"fold1", "fold2", "fold3", "fold4", "fold5"}
        if not expected_folds.issubset(set(obj.keys())):
            warnings.warn(f"{fp} does not contain the expected folds; skipping.")
            continue
        data[dataset_name] = obj
    return data

def to_1d(y):
    """Ensure y is a 1D array-like."""
    if y is None:
        return None
    y = np.asarray(y)
    return y.ravel()

def rename_duplicates(old_columns):
    # Dictionary to count occurrences of each column name
    counts = {}
    new_columns = []
    for column in old_columns:
        if column in counts:
            counts[column] += 1
            new_columns.append(f"{column}_{counts[column]}")
        else:
            counts[column] = 0
            new_columns.append(column)
    return new_columns

Model Fitting Functions

In [3]:
# -----------------------------
# Phase 1: Train and Save Predictions
# -----------------------------
def generate_and_save_predictions():
    """
    For each method folder, train all models per dataset/fold and
    save predictions + y_true into one pickle per method.
    Pickle schema:
    { dataset_name:
        { fold_name:
            { model_name: {"y_true": np.ndarray, "y_pred": np.ndarray} }
        }
    }
    """
    models = build_models()
    for method in METHOD_FOLDERS:
        print(f"\n=== Processing method: {method} ===")
        method_data = load_method_datasets(method)
        method_pred_store = {}

        for dataset_name, folds_dict in method_data.items():
            print(f"  - Dataset: {dataset_name}")
            method_pred_store[dataset_name] = {}

            for fold_name, fold_payload in folds_dict.items():
                # Expected keys: Training_Independent, Training_Dependent, Testing_Independent, Testing_Dependent, Timing
                X_tr = fold_payload.get("Training_Independent")
                y_tr = to_1d(fold_payload.get("Training_Dependent"))
                X_te = fold_payload.get("Testing_Independent")
                y_te = to_1d(fold_payload.get("Testing_Dependent"))

                X_tr.columns = [col.replace('{', '').replace('}', '').replace(':', '').replace('"', '').replace("'", '').replace('[', '').replace(']', '').replace(')', '').replace(',', '_').replace('.', '_').replace(' ', '').replace('(','_')  for col in X_tr.columns]
                X_te.columns = [col.replace('{', '').replace('}', '').replace(':', '').replace('"', '').replace("'", '').replace('[', '').replace(']', '').replace(')', '').replace(',', '_').replace('.', '_').replace(' ', '').replace('(','_') for col in X_te.columns]
                # Renaming duplicated columns
                X_tr.columns = rename_duplicates(X_tr.columns)
                X_te.columns = rename_duplicates(X_te.columns)

                if X_tr is None or y_tr is None or X_te is None or y_te is None:
                    warnings.warn(f"Missing data in {method}/{dataset_name}/{fold_name}; skipping this fold.")
                    continue

                method_pred_store[dataset_name][fold_name] = {}

                for model_name, model in models.items():
                    # Recreate model fresh for each fit (avoid state carry-over)
                    # For pipelines, clone by rebuilding from build_models()
                    model_fresh = build_models()[model_name]
                    try:
                        model_fresh.fit(X_tr, y_tr)
                        y_pred = model_fresh.predict(X_te)
                    except Exception as e:
                        warnings.warn(f"Model {model_name} failed on {method}/{dataset_name}/{fold_name}: {e}")
                        y_pred = np.full_like(y_te, fill_value=np.nan, dtype=float)

                    method_pred_store[dataset_name][fold_name][model_name] = {
                        "y_true": np.asarray(y_te),
                        "y_pred": np.asarray(y_pred, dtype=float),
                    }

        # Write out per-method predictions
        out_fp = os.path.join(PREDICTIONS_DIR, f"{method}_predictions.pkl")
        with open(out_fp, "wb") as f:
            pickle.dump(method_pred_store, f, protocol=pickle.HIGHEST_PROTOCOL)
        print(f"Saved predictions for {method} -> {out_fp}")

Computing Metrics

In [4]:
# -----------------------------
# Phase 2: Reload and Compute RMSE (averaged across 5 folds)
# -----------------------------
def compute_rmse_from_saved_predictions():
    """
    Reads each method's prediction pickle and computes:
    Test RMSE averaged across folds per (Dataset, Model, Method).

    Returns a wide DataFrame:
    columns = ['Dataset', 'Model', 'AutoFeat', 'SAFE', 'FT', 'OpenFE']
    """
    # Load all method prediction pickles
    method_to_preds = {}
    for method in METHOD_FOLDERS:
        pred_fp = os.path.join(PREDICTIONS_DIR, f"{method}_predictions.pkl")
        if not os.path.exists(pred_fp):
            warnings.warn(f"Predictions file not found for {method}: {pred_fp}.")
            continue
        with open(pred_fp, "rb") as f:
            method_to_preds[method] = pickle.load(f)

    # Aggregate RMSE per dataset, per model, per method across folds
    rows = []
    for method, all_datasets in method_to_preds.items():
        for dataset_name, folds_dict in all_datasets.items():
            # model_name -> list of fold RMSEs
            model_to_fold_rmses = {}

            for fold_name, models_dict in folds_dict.items():
                for model_name, payload in models_dict.items():
                    y_true = payload.get("y_true", None)
                    y_pred = payload.get("y_pred", None)
                    if y_true is None or y_pred is None or len(y_true) == 0:
                        continue
                    # Skip NaN predictions
                    if np.isnan(y_pred).any():
                        continue
                    rmse = mean_squared_error(y_true, y_pred)**0.5
                    model_to_fold_rmses.setdefault(model_name, []).append(rmse)

            # Average across folds
            for model_name, rmse_list in model_to_fold_rmses.items():
                if len(rmse_list) == 0:
                    continue
                avg_rmse = float(np.mean(rmse_list))
                rows.append({
                    "Dataset": dataset_name,
                    "Model": model_name,
                    "Method": method,
                    "Test_RMSE": avg_rmse,
                })

    # Long -> wide
    df_long = pd.DataFrame(rows)

    return df_long


Run the methods

In [5]:
# # -----------------------------
# # Driver
# # -----------------------------

# # Phase 1: train and save predictions
# generate_and_save_predictions()


In [6]:

# Phase 2: reload and compute RMSE table
df_long = compute_rmse_from_saved_predictions()

# Save outputs for convenience

# Also pickle the DataFrame if you want to load it later
with open(os.path.join(PREDICTIONS_DIR, "rmse_averaged.pkl"), "wb") as f:
    pickle.dump(df_long, f, protocol=pickle.HIGHEST_PROTOCOL)

Renaming the datasets

In [7]:
# Remove '_fold_data' from Dataset column
df_long['Dataset'] = df_long['Dataset'].str.replace('_fold_data', '', regex=False)

Adding the Best K Predictions

In [8]:
import pickle
import pandas as pd

# Load the pickled df_summary
with open('df_summary.pkl', 'rb') as f:
    df_summary_loaded = pickle.load(f)

# Append df_summary to df_long
df_long = pd.concat([df_long, df_summary_loaded], ignore_index=True)

print(df_long)


                 Dataset                                  Model Method  \
0     Airfoil_Self_Noise                      Linear Regression  BASIC   
1     Airfoil_Self_Noise                         KNN Regression  BASIC   
2     Airfoil_Self_Noise                         SVM Regression  BASIC   
3     Airfoil_Self_Noise               Random Forest Regression  BASIC   
4     Airfoil_Self_Noise                    AdaBoost Regression  BASIC   
...                  ...                                    ...    ...   
1293               Quake               Decision Tree Regression    EFS   
1294               Quake  Extremely Randomized Trees Regression    EFS   
1295               Quake           Gradient Boosting Regression    EFS   
1296               Quake                    LightGBM Regression    EFS   
1297               Quake                     XGBoost Regression    EFS   

      Test_RMSE  
0      4.850325  
1      3.578602  
2      4.235507  
3      1.826100  
4      3.878246  
...

Reshaping the dataframe

In [9]:
# Pivot the dataframe
df_wide = df_long.pivot_table(
    index=['Dataset', 'Model'],   # Keep these as identifiers
    columns='Method',             # Each method becomes a column
    values='Test_RMSE'            # Fill with Test_RMSE values
).reset_index()

# Optional: ensure the column order you mentioned
method_order = ['BASIC', 'AUTOFEAT', 'FT', 'SAFE', 'EFS', 'OPENFE']
df_wide = df_wide[['Dataset', 'Model'] + method_order]

print(df_wide)


Method             Dataset                                  Model     BASIC  \
0       Airfoil_Self_Noise                    AdaBoost Regression  3.878246   
1       Airfoil_Self_Noise               Decision Tree Regression  2.658327   
2       Airfoil_Self_Noise  Extremely Randomized Trees Regression  1.548887   
3       Airfoil_Self_Noise           Gradient Boosting Regression  2.684646   
4       Airfoil_Self_Noise                         KNN Regression  3.578602   
..                     ...                                    ...       ...   
226                  pyrim                      Linear Regression  0.166263   
227                  pyrim                         MLP Regression  0.162682   
228                  pyrim               Random Forest Regression  0.089179   
229                  pyrim                         SVM Regression  0.096645   
230                  pyrim                     XGBoost Regression  0.088489   

Method  AUTOFEAT        FT      SAFE       EFS    O

Removing Fertility as it is a classification Dataset

In [10]:
# Remove rows where Dataset contains 'Fertility'
df_wide = df_wide[~df_wide['Dataset'].str.contains('Fertility', case=False, na=False)]

Visualizing the Comparison

In [11]:
import pandas as pd
import matplotlib.pyplot as plt

methods_to_compare = ['BASIC', 'AUTOFEAT', 'FT', 'SAFE', 'OPENFE']

def classify_outcome(row, compare_method):
    if row['EFS'] < row[compare_method]:
        return "Improved"
    elif row['EFS'] == row[compare_method]:
        return "Same"
    else:
        return "Decreased"

for method in methods_to_compare:
    # Drop rows where either EFS or the method is NaN
    temp_df = df_wide.dropna(subset=['EFS', method]).copy()
    temp_df['Outcome'] = temp_df.apply(lambda row: classify_outcome(row, method), axis=1)

    counts = (
        temp_df.groupby(['Model', 'Outcome'])
               .size()
               .reset_index(name='Count')
    )

    total_per_model = counts.groupby('Model')['Count'].transform('sum')
    counts['Percentage'] = (counts['Count'] / total_per_model) * 100

    # Pivot and ensure all outcome columns exist
    plot_df = counts.pivot(index='Model', columns='Outcome', values='Percentage').fillna(0)
    for col in ["Improved", "Same", "Decreased"]:
        if col not in plot_df.columns:
            plot_df[col] = 0
    plot_df = plot_df[["Improved", "Same", "Decreased"]]  # enforce column order

    # Plot
    plot_df.plot(kind='bar', stacked=False)
    plt.ylabel("Percentage (%)")
    plt.xlabel("Model")
    plt.xticks(rotation=90)  # Rotate x-axis labels vertically
    plt.legend(title="Outcome", loc="upper left", bbox_to_anchor=(1.05, 1))
    plt.ylim(0, 100)
    plt.tight_layout()

    # Save as PDF (lowercase method name in filename, spaces replaced with underscores)
    filename = f"{method.lower()}_comparison.pdf".replace(" ", "_")
    plt.savefig(filename, format="pdf", bbox_inches="tight")

    plt.close()  # Close the figure to free memory


Comparing AFE against Basic

In [12]:
import pandas as pd
import matplotlib.pyplot as plt

methods_to_compare = ['AUTOFEAT', 'FT', 'SAFE', 'OPENFE']

def classify_outcome(row, compare_method):
    if row['BASIC'] > row[compare_method]:
        return "Improved"
    elif row['BASIC'] == row[compare_method]:
        return "Same"
    else:
        return "Decreased"

for method in methods_to_compare:
    # Drop rows where either EFS or the method is NaN
    temp_df = df_wide.dropna(subset=['EFS', method]).copy()
    temp_df['Outcome'] = temp_df.apply(lambda row: classify_outcome(row, method), axis=1)

    counts = (
        temp_df.groupby(['Model', 'Outcome'])
               .size()
               .reset_index(name='Count')
    )

    total_per_model = counts.groupby('Model')['Count'].transform('sum')
    counts['Percentage'] = (counts['Count'] / total_per_model) * 100

    # Pivot and ensure all outcome columns exist
    plot_df = counts.pivot(index='Model', columns='Outcome', values='Percentage').fillna(0)
    for col in ["Improved", "Same", "Decreased"]:
        if col not in plot_df.columns:
            plot_df[col] = 0
    plot_df = plot_df[["Improved", "Same", "Decreased"]]  # enforce column order

    # Plot
    plot_df.plot(kind='bar', stacked=False)
    plt.ylabel("Percentage (%)")
    plt.xlabel("Model")
    plt.xticks(rotation=90)  # Rotate x-axis labels vertically
    plt.legend(title="Outcome", loc="upper left", bbox_to_anchor=(1.05, 1))
    plt.ylim(0, 100)
    plt.tight_layout()

    # Save as PDF (lowercase method name in filename, spaces replaced with underscores)
    filename = f"{method.lower()}_basic_comparison.pdf".replace(" ", "_")
    plt.savefig(filename, format="pdf", bbox_inches="tight")

    plt.close()  # Close the figure to free memory


Checking for significant differences

In [22]:
import pandas as pd
from scipy.stats import wilcoxon

def compare_basic_lt_efs(df: pd.DataFrame) -> pd.DataFrame:
    """
    Run Wilcoxon signed-rank test per model to check if BASIC < EFS.
    """
    results = []
    for model in df['Model'].unique():
        subset = df[df['Model'] == model]

        # Paired values
        basic_vals = subset['BASIC'].values
        efs_vals = subset['EFS'].values

        # One-sided test: BASIC < EFS
        stat, p_value = wilcoxon(basic_vals, efs_vals, alternative='less')

        results.append({
            'Model': model,
            'Wilcoxon_stat': stat,
            'p_value': p_value,
            'Significant_(p<0.05)': p_value < 0.05
        })

    return pd.DataFrame(results)

# Example usage
df_results = compare_basic_lt_efs(df_wide)
print(df_results)


                                    Model  Wilcoxon_stat   p_value  \
0                     AdaBoost Regression           98.0  0.406178   
1                Decision Tree Regression          112.0  0.753049   
2   Extremely Randomized Trees Regression          102.0  0.610911   
3            Gradient Boosting Regression           95.0  0.809418   
4                          KNN Regression          168.0  0.992344   
5                     LightGBM Regression           79.0  0.388559   
6                       Linear Regression           71.0  0.561639   
7                          MLP Regression          142.0  0.993065   
8                Random Forest Regression           99.0  0.563941   
9                          SVM Regression          133.0  0.999612   
10                     XGBoost Regression           78.0  0.371975   

    Significant_(p<0.05)  
0                  False  
1                  False  
2                  False  
3                  False  
4                  False

Checking globally

In [23]:
from scipy.stats import wilcoxon

# Paired samples
basic_vals = df_wide['BASIC'].values
efs_vals = df_wide['EFS'].values

# One-sided Wilcoxon: test if EFS < BASIC
stat, p_value = wilcoxon(efs_vals, basic_vals, alternative='less')

# Put results in a dataframe
df_results = pd.DataFrame([{
    'Comparison': 'EFS vs BASIC',
    'Wilcoxon_stat': stat,
    'p_value': p_value,
    'Significant': p_value < 0.05
}])

print(df_results)


     Comparison  Wilcoxon_stat   p_value  Significant
0  EFS vs BASIC         7046.0  0.000123         True


Checking for AFE

In [24]:
import numpy as np
import pandas as pd
from scipy.stats import wilcoxon

def wilcoxon_basic_lt_method_by_model(df: pd.DataFrame, method_col: str, alpha: float = 0.05) -> pd.DataFrame:
    """
    For each Model, run a one-sided Wilcoxon signed-rank test to check if BASIC < AFE method.
    - Pairs are (BASIC, method_col) across datasets within each Model.
    - NaN rows are dropped per-model.
    - If all paired differences are zero or there are fewer than 1 valid pairs, returns NaNs for stat/p.
    """
    rows = []
    for model, sub in df.groupby('Model', dropna=False):
        # Keep only rows where both columns are present
        paired = sub[['BASIC', method_col]].dropna()

        # Count pairs
        n_pairs = len(paired)
        stat = np.nan
        pval = np.nan
        significant = False
        n_zero = 0

        if n_pairs > 0:
            x = paired['BASIC'].to_numpy()
            y = paired[method_col].to_numpy()
            diffs = x - y
            n_zero = int(np.sum(diffs == 0))

            # Need at least one non-zero difference for Wilcoxon to run
            if np.any(diffs != 0):
                stat, pval = wilcoxon(x, y, alternative='less', zero_method='wilcox')
                significant = bool(pval < alpha)

        rows.append({
            'Model': model,
            'Method': method_col,
            'n_pairs': n_pairs,
            'n_zero_diffs': n_zero,
            'Wilcoxon_stat': stat,
            'p_value': pval,
            'Significant_(p<{})'.format(alpha): significant
        })

    return pd.DataFrame(rows).sort_values('Model').reset_index(drop=True)

# ---- Run for each AFE method (BASIC < METHOD) ----
df_res_AUTOF   = wilcoxon_basic_lt_method_by_model(df_wide, 'AUTOFEAT')
df_res_FT      = wilcoxon_basic_lt_method_by_model(df_wide, 'FT')
df_res_SAFE    = wilcoxon_basic_lt_method_by_model(df_wide, 'SAFE')
df_res_OPENFE  = wilcoxon_basic_lt_method_by_model(df_wide, 'OPENFE')
df_EFS         = wilcoxon_basic_lt_method_by_model(df_wide, 'EFS')


# Optional: display / inspect
for name, d in [('AUTOFEAT', df_res_AUTOF),
                ('FT', df_res_FT),
                ('SAFE', df_res_SAFE),
                ('OPENFE', df_res_OPENFE),
                ('EFS', df_EFS)]:
    print(f"\n=== BASIC < {name} (per model) ===")
    print(d)



=== BASIC < AUTOFEAT (per model) ===
                                    Model    Method  n_pairs  n_zero_diffs  \
0                     AdaBoost Regression  AUTOFEAT       19             0   
1                Decision Tree Regression  AUTOFEAT       19             0   
2   Extremely Randomized Trees Regression  AUTOFEAT       19             0   
3            Gradient Boosting Regression  AUTOFEAT       19             0   
4                          KNN Regression  AUTOFEAT       19             0   
5                     LightGBM Regression  AUTOFEAT       19             0   
6                       Linear Regression  AUTOFEAT       19             0   
7                          MLP Regression  AUTOFEAT       19             0   
8                Random Forest Regression  AUTOFEAT       19             0   
9                          SVM Regression  AUTOFEAT       19             0   
10                     XGBoost Regression  AUTOFEAT       19             0   

    Wilcoxon_stat   p_val